In [1]:
# import relevant libraries
import numpy as np
import pandas as pd
from collections import deque
from datetime import datetime
import os
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import ast

## 1: Getting logs and visibility dictionnary

We will want to add the visibility metric to the sessions. For that we start by concatenating all the arks_counts dataframes and summing the counts by ark. We then turn it into a dictionnary for quick lookup later.

In [3]:
# get ark counts in one dataframe

arks_dir = 'data_temp_month/unique_arks'
arks_files = [f for f in os.listdir(arks_dir) if f.endswith('.csv')]
# init empty df
combined_arks_counts = pd.DataFrame()

# read and concatenate all unique_arks_counts files
for file in arks_files:
    file_path = os.path.join(arks_dir, file)
    df = pd.read_csv(file_path)
    combined_arks_counts = pd.concat([combined_arks_counts, df])

In [4]:
combined_arks_counts.shape

(10944349, 2)

In [5]:
# group by ark and sum the counts
final_arks_counts = combined_arks_counts.groupby('Ark', as_index=False).sum()

In [6]:
final_arks_counts[270:290]

,Ark,Count
270,bpt6k100020v,13
271,bpt6k100022m,3
272,bpt6k1000230,1
273,bpt6k100024c,2
274,bpt6k100025r,3
275,bpt6k1000264,2
276,bpt6k100027h,6
277,bpt6k1000298,8
278,bpt6k10002v,14
279,bpt6k1000306,1


In [7]:
# save the combined DataFrame to a new CSV file
final_arks_counts.to_csv('arks_temp_month/final_arks_counts.csv', index=False)

In [8]:
# create dictionnary for quick lookup later
ark_count_dict = final_arks_counts.set_index('Ark')['Count'].to_dict()

In [9]:
# function to get the visibility of a certain ark
def get_visibility(ark_list):
    return [ark_count_dict.get(ark, 0) for ark in ark_list]

In [10]:
# extracting logs to sessionize
logs_df = pd.read_csv("data_temp_month/clean_logs0.csv")

## 2: Sessionization

Create and enrich user sessions. The goal will then be to classify these sessions and find the ones relating to a "Rabbit Hole".

In [11]:
# check all relevant columns are there
print(logs_df.columns)

Index(['0', 'IPaddress', 'Date', 'Request', 'Referrer', 'Ark', 'search_terms'], dtype='object')


From these logs, we want to create user sessions, so we begin by grouping the rows by IP address and aggregating the features.

In [12]:
# group by ip address 
sessions_df = logs_df.groupby('IPaddress').agg({'Ark':list, 'Date':list, 'search_terms':list, 'Referrer':list})

In [13]:
sessions_df

,Ark,Date,search_terms,Referrer
IPaddress,,,,
1000d74feba89503094f7352a39cea90,"[nan, nan]","[2016-02-27 20:49:48+01:00, 2016-02-27 20:59:4...","[[], []]","[-, -]"
10010a4a2c394dd20ac1cece871a1d7b,"[bpt6k5531462t, nan, nan, nan, nan, nan, nan, ...","[2016-02-29 05:02:52+01:00, 2016-02-29 05:02:5...","[[], [], [], [], [], [], [], [], [], [], [], [...","[https://www.google.es/, http://gallica.bnf.fr..."
1002ad1e79476f955b17fc2dda72ffb1,"[btv1b86108062, nan, nan, nan, nan, nan, nan, ...","[2016-02-27 20:16:15+01:00, 2016-02-27 20:16:1...","[[], [], [], [], [], [], [], [], [], [], [], [...",[http://images.google.fr/imgres?imgurl=http%3A...
1002f46d4d4bdd5ae010bf8e92e9ed22,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2016-02-27 18:35:21+01:00, 2016-02-27 18:35:2...","[[], [], [], [], [], [], [], [], [], [], [], [...",[http://www.google.fr/url?sa=t&rct=j&q=&esrc=s...
1003e3f730f45ed76f67666be0867c5a,"[nan, nan, nan, nan, nan, nan, nan, nan]","[2016-02-28 03:53:19+01:00, 2016-02-28 03:53:2...","[['Auriol, George', 'Auriol, George', 'papiers...","[-, -, -, -, -, -, -, -]"
...,...,...,...,...
fff9a541a90c0763044cde602f6c81eb,"[bpt6k824366, bpt6k824366, nan, nan, nan, nan,...","[2016-02-28 04:48:58+01:00, 2016-02-28 04:48:5...","[[], [], [], [], [], [], [], [], [], [], [], [...","[http://www.lexilogos.com/afrique_langues.htm,..."
fffa7bea05a91377dc743b11be2fe2a6,"[btv1b55008737g, nan, nan, nan, nan, nan, nan,...","[2016-02-28 19:35:50+01:00, 2016-02-28 19:35:5...","[[], [], [], [], [], [], [], [], [], [], [], [...",[http://gallica.bnf.fr/html/und/cartes/les-glo...
fffb71b741dfd9453af0fad36b9c8571,"[bpt6k6528902c, nan, nan, nan, nan, nan, nan, ...","[2016-02-28 16:45:55+01:00, 2016-02-28 16:45:5...","[[], [], [], [], [], [], [], [], [], [], [], [...",[http://www.geneanet.org/archives/ouvrages/?ac...


We want to compute the time between two connections following one another.

In [14]:
# function to compute minutes between two dates
def minutes_between(d1, d2):
    # Parse the dates using the appropriate format
    d1 = datetime.strptime(d1, "%Y-%m-%d %H:%M:%S")
    d2 = datetime.strptime(d2, "%Y-%m-%d %H:%M:%S")
    
    # Calculate the difference in minutes
    return abs((d2 - d1).total_seconds() // 60)

In [15]:
time_beginning = "01/Jan/0001:01:01:01 +0100"
time_end = "01/Jan/3000:01:01:01 +0100"
sessions_df['date_1'] = sessions_df.apply(lambda x: [time_beginning]+x['Date'], axis = 1)
sessions_df['date_2'] = sessions_df.apply(lambda x: x['Date']+[time_end],axis=1)

In [16]:
# function to calculate the difference between two zipped lists
def calculate_difference_zipped_list(lst):
    new_lst = []
    for e in lst:
        if (e[0]==time_beginning):
            new_lst.append(999)
        elif (e[1]==time_end):
            new_lst.append(999)
        else:
            new_lst.append(minutes_between(e[0][:-6], e[1][:-6]))
    return new_lst
        
# 999 means this is the first connection    

In [17]:
# this contains the ip addresses and the zipped version of date_1, date_2
IP_and_sessions = sessions_df.apply(lambda x: deque(calculate_difference_zipped_list(list(zip(x['date_1'],x['date_2'])))),axis=1)

In [18]:
# IP and the difference in time between each connection and the last
IP_and_sessions

IPaddress
1000d74feba89503094f7352a39cea90                                     [999, 10.0, 999]
10010a4a2c394dd20ac1cece871a1d7b    [999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1002ad1e79476f955b17fc2dda72ffb1    [999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1002f46d4d4bdd5ae010bf8e92e9ed22    [999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1003e3f730f45ed76f67666be0867c5a        [999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 999]
                                                          ...                        
fff9a541a90c0763044cde602f6c81eb    [999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
fffa7bea05a91377dc743b11be2fe2a6    [999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
fffb71b741dfd9453af0fad36b9c8571    [999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
fffc03fa5d786e56233c7f8c70ddb5c5                                           [999, 999]
ffff335e139e53a8b99899283caf5c3d                                      [999, 0.0, 999]
Length: 51822, dtype: object

In [19]:
# create previous connexion date feature, to have the last known session date for each IP address
logs_df['previous_connexion_date'] = logs_df.apply(lambda x: IP_and_sessions[x['IPaddress']].popleft(),axis=1)

In [20]:
logs_df.head(4)

,0,IPaddress,Date,Request,Referrer,Ark,search_terms,previous_connexion_date
0,NaN,3c2af9233e11938ca3f73eb650d4af40,2016-02-27 18:01:04+01:00,GET /ark:/12148/bpt6k57843235,http://www.google.fr/url?sa=t&rct=j&q=&esrc=s&...,bpt6k57843235,[],999.0
1,NaN,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /ark:/12148/btv1b8490545v,http://images.google.de/imgres?imgurl=http://g...,btv1b8490545v,[],999.0
2,NaN,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/vendor/bootstra...,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,NaN,[],0.0
3,NaN,c6bd521083f4402a71e65c33baa00f3e,2016-02-27 18:01:05+01:00,GET /assets/static/stylesheets/main.css,http://gallica.bnf.fr/ark:/12148/btv1b8490545v,NaN,[],0.0


We generate session IDs based on period, a new session ID is generated if the period between the last connexion and this one is > 60 minutes.
Trying different values than 60 yields little change. 

In [21]:
session_id = 0

def create_session(period, max_time):
    global session_id
    if(period > max_time):
        session_id += 1
    return session_id

In [22]:
# sort by ip address and date
logs_df = logs_df.sort_values(by=['IPaddress', 'Date'])
# create session ids, a new session is created if the previous connexion was more than 60 minutes ago
logs_df['session_id'] = logs_df.apply(lambda x: create_session(x['previous_connexion_date'], 60),axis=1)

In [23]:
logs_df

,0,IPaddress,Date,Request,Referrer,Ark,search_terms,previous_connexion_date,session_id
2539722,NaN,1000d74feba89503094f7352a39cea90,2016-02-27 20:49:48+01:00,GET /favicon.ico,-,NaN,[],999.0,1
2543530,NaN,1000d74feba89503094f7352a39cea90,2016-02-27 20:59:48+01:00,GET /favicon.ico,-,NaN,[],10.0,1
12057797,NaN,10010a4a2c394dd20ac1cece871a1d7b,2016-02-29 05:02:52+01:00,GET /ark:/12148/bpt6k5531462t,https://www.google.es/,bpt6k5531462t,[],999.0,2
12057798,NaN,10010a4a2c394dd20ac1cece871a1d7b,2016-02-29 05:02:53+01:00,GET /assets/static/stylesheets/persoScrollBar.css,http://gallica.bnf.fr/ark:/12148/bpt6k5531462t,NaN,[],0.0,2
12057799,NaN,10010a4a2c394dd20ac1cece871a1d7b,2016-02-29 05:02:53+01:00,GET /assets/static/stylesheets/main.css,http://gallica.bnf.fr/ark:/12148/bpt6k5531462t,NaN,[],0.0,2
...,...,...,...,...,...,...,...,...,...
14064220,NaN,fffb71b741dfd9453af0fad36b9c8571,2016-02-28 16:51:02+01:00,"GET /iiif/ark:/12148/bpt6k6528902c/f48/2816,48...",http://gallica.bnf.fr/ark:/12148/bpt6k6528902c...,bpt6k6528902c,[],0.0,70373
14064221,NaN,fffb71b741dfd9453af0fad36b9c8571,2016-02-28 16:51:02+01:00,"GET /iiif/ark:/12148/bpt6k6528902c/f48/1024,48...",http://gallica.bnf.fr/ark:/12148/bpt6k6528902c...,bpt6k6528902c,[],0.0,70373
10026987,NaN,fffc03fa5d786e56233c7f8c70ddb5c5,2016-02-29 03:04:07+01:00,GET /scripts/get_page.exe?O=3144&E=908&N=3&CD=...,-,NaN,[],999.0,70374
2116086,NaN,ffff335e139e53a8b99899283caf5c3d,2016-02-27 19:00:46+01:00,GET /ark:/12148/bpt6k409909g/texteBrut,https://www.google.fr/,bpt6k409909g,[],999.0,70375


In [24]:
#create sessions by grouping by session ID and collecting all arks and their metadata
sessions = logs_df.groupby('session_id').agg({'Ark':list,'Date':list, 'Referrer':list, 'search_terms':list})

In [25]:
# removing subsequent ARKs and removing empty lists
def remove_consecutive_duplicates(l):
    return [v for i, v in enumerate(l) if (i == 0 or v != l[i-1]) and v!=[]]

    
sessions['Ark'] = sessions.apply(lambda x: remove_consecutive_duplicates(x['Ark']), axis = 1)

In [26]:
# keep only first and last dates
sessions['Date'] = sessions['Date'].apply(lambda x: [x[0],x[-1]])

In [27]:
# keep only first clean referrer
def get_first_referrer(referrers):
    if isinstance(referrers, list) and referrers:  # Check if referrers is a non-empty list
        first_referrer = referrers[0]
        if isinstance(first_referrer, str):  # Check if the first referrer is a string
            parsed_url = urlparse(first_referrer)
            return f"{parsed_url.scheme}://{parsed_url.netloc}"
    return None

In [28]:
# Apply the function to the 'Referrer' column to create 'first_referrer'
sessions['first_referrer'] = sessions['Referrer'].apply(get_first_referrer)

In [29]:
sessions

,Ark,Date,Referrer,search_terms,first_referrer
session_id,,,,,
1,"[nan, nan]","[2016-02-27 20:49:48+01:00, 2016-02-27 20:59:4...","[-, -]","[[], []]",://
2,"[bpt6k5531462t, nan, nan, nan, nan, nan, nan, ...","[2016-02-29 05:02:52+01:00, 2016-02-29 05:03:2...","[https://www.google.es/, http://gallica.bnf.fr...","[[], [], [], [], [], [], [], [], [], [], [], [...",https://www.google.es
3,"[btv1b86108062, nan, nan, nan, nan, nan, nan, ...","[2016-02-27 20:16:15+01:00, 2016-02-27 20:16:2...",[http://images.google.fr/imgres?imgurl=http%3A...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://images.google.fr
4,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2016-02-27 18:35:21+01:00, 2016-02-27 18:49:3...",[http://www.google.fr/url?sa=t&rct=j&q=&esrc=s...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://www.google.fr
5,"[nan, nan, nan, nan, nan, nan, nan, nan]","[2016-02-28 03:53:19+01:00, 2016-02-28 03:53:2...","[-, -, -, -, -, -, -, -]","[['Auriol, George', 'Auriol, George', 'papiers...",://
...,...,...,...,...,...
70371,"[bpt6k824366, nan, nan, nan, nan, nan, nan, na...","[2016-02-28 04:48:58+01:00, 2016-02-28 04:49:1...","[http://www.lexilogos.com/afrique_langues.htm,...","[[], [], [], [], [], [], [], [], [], [], [], [...",http://www.lexilogos.com
70372,"[btv1b55008737g, nan, nan, nan, nan, nan, nan,...","[2016-02-28 19:35:50+01:00, 2016-02-28 19:36:3...",[http://gallica.bnf.fr/html/und/cartes/les-glo...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://gallica.bnf.fr
70373,"[bpt6k6528902c, nan, nan, nan, nan, nan, nan, ...","[2016-02-28 16:45:55+01:00, 2016-02-28 16:51:0...",[http://www.geneanet.org/archives/ouvrages/?ac...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://www.geneanet.org


In [30]:
# function to find the length of a session
def length_session(d1, d2):
    # Remove the colon from the timezone offset
    d1 = d1[:-3] + d1[-2:]
    d2 = d2[:-3] + d2[-2:]
    
    # Parse the dates using the appropriate format
    d1 = datetime.strptime(d1, "%Y-%m-%d %H:%M:%S%z")
    d2 = datetime.strptime(d2, "%Y-%m-%d %H:%M:%S%z")
    
    # Calculate the difference in minutes
    return abs((d2 - d1).total_seconds() // 60)

In [31]:
# add sessions length in minutes feature
sessions['length_minutes'] = sessions['Date'].apply(lambda x: length_session(x[0], x[-1]))

In [32]:
# add a list of visibilities for each ark
sessions['visibility'] = sessions['Ark'].apply(get_visibility)

We create many visibility features : the min and mean visibility, the min/mean of the first 3 and last 3 arks, and the variation between these. The purpose of this is to see if the visbility decreases at the end of a session or increases.

In [33]:
sessions = sessions.assign(
    min_visibility=lambda x: x['visibility'].apply(lambda vis: min(filter(None, vis), default=0)),
    mean_visibility=lambda x: x['visibility'].apply(lambda vis: sum(vis) / len([v for v in vis if v != 0]) if any(vis) else 0),
    min_first_3=lambda x: x['visibility'].apply(lambda vis: min(filter(None, vis[:3]), default=0)),
    mean_first_3=lambda x: x['visibility'].apply(lambda vis: sum(vis[:3]) / len([v for v in vis[:3] if v != 0]) if any(vis[:3]) else 0),
    min_last_3=lambda x: x['visibility'].apply(lambda vis: min(filter(None, vis[-3:]), default=0)),
    mean_last_3=lambda x: x['visibility'].apply(lambda vis: sum(vis[-3:]) / len([v for v in vis[-3:] if v != 0]) if any(vis[-3:]) else 0),
    variation_min_vis=lambda x: x['min_last_3'] - x['min_first_3'],
    variation_mean_vis=lambda x: x['mean_last_3'] - x['mean_first_3']
)

In [34]:
sessions

,Ark,Date,Referrer,search_terms,first_referrer,length_minutes,visibility,min_visibility,mean_visibility,min_first_3,mean_first_3,min_last_3,mean_last_3,variation_min_vis,variation_mean_vis
session_id,,,,,,,,,,,,,,,
1,"[nan, nan]","[2016-02-27 20:49:48+01:00, 2016-02-27 20:59:4...","[-, -]","[[], []]",://,10.0,"[0, 0]",0,0.00000,0,0.0,0,0.000000,0,0.000000
2,"[bpt6k5531462t, nan, nan, nan, nan, nan, nan, ...","[2016-02-29 05:02:52+01:00, 2016-02-29 05:03:2...","[https://www.google.es/, http://gallica.bnf.fr...","[[], [], [], [], [], [], [], [], [], [], [], [...",https://www.google.es,0.0,"[296, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",296,296.00000,296,296.0,296,296.000000,0,0.000000
3,"[btv1b86108062, nan, nan, nan, nan, nan, nan, ...","[2016-02-27 20:16:15+01:00, 2016-02-27 20:16:2...",[http://images.google.fr/imgres?imgurl=http%3A...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://images.google.fr,0.0,"[516, 0, 0, 0, 0, 0, 0, 516, 0, 0, 0, 0, 0, 0,...",516,516.00000,516,516.0,516,516.000000,0,0.000000
4,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2016-02-27 18:35:21+01:00, 2016-02-27 18:49:3...",[http://www.google.fr/url?sa=t&rct=j&q=&esrc=s...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://www.google.fr,14.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,1697.61658,0,0.0,15,713.666667,15,713.666667
5,"[nan, nan, nan, nan, nan, nan, nan, nan]","[2016-02-28 03:53:19+01:00, 2016-02-28 03:53:2...","[-, -, -, -, -, -, -, -]","[['Auriol, George', 'Auriol, George', 'papiers...",://,0.0,"[0, 0, 0, 0, 0, 0, 0, 0]",0,0.00000,0,0.0,0,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70371,"[bpt6k824366, nan, nan, nan, nan, nan, nan, na...","[2016-02-28 04:48:58+01:00, 2016-02-28 04:49:1...","[http://www.lexilogos.com/afrique_langues.htm,...","[[], [], [], [], [], [], [], [], [], [], [], [...",http://www.lexilogos.com,0.0,"[248, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",4,609.06250,248,248.0,95,498.500000,-153,250.500000
70372,"[btv1b55008737g, nan, nan, nan, nan, nan, nan,...","[2016-02-28 19:35:50+01:00, 2016-02-28 19:36:3...",[http://gallica.bnf.fr/html/und/cartes/les-glo...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://gallica.bnf.fr,0.0,"[3662, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3662,3662.00000,3662,3662.0,0,0.000000,-3662,-3662.000000
70373,"[bpt6k6528902c, nan, nan, nan, nan, nan, nan, ...","[2016-02-28 16:45:55+01:00, 2016-02-28 16:51:0...",[http://www.geneanet.org/archives/ouvrages/?ac...,"[[], [], [], [], [], [], [], [], [], [], [], [...",http://www.geneanet.org,5.0,"[35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",35,35.00000,35,35.0,35,35.000000,0,0.000000


In [35]:
# save temporary sessions
sessions.to_csv("data_temp_month/sessions/sessions0.csv")

## 3: Adding features to sessions to later help characterize rabbit holes
To help characterize rabbit holes we add features such as theme, type. We begin by concatenanting all the previously obtained sessions into one.

In [39]:
# concatenating the sessions
sessions_dir = 'data_temp_month/sessions'
sessions_files = [f for f in os.listdir(sessions_dir) if f.endswith('.csv')]
# init empty df
combined_sessions = pd.DataFrame()

# read and concatenate all sessions files
for file in tqdm(sessions_files, desc="Combining session files"):
    file_path = os.path.join(sessions_dir, file)
    df = pd.read_csv(file_path)
    combined_sessions = pd.concat([combined_sessions, df])

Combining session files: 100%|██████████████████| 22/22 [03:48<00:00, 10.37s/it]


In [40]:
combined_sessions.shape

(1417035, 16)

In [90]:
# function to make sure the arks is a list of strings
def convert_ark_string(ark_string):
    try:
        # Replace nan (unquoted) with 'nan' (quoted)
        ark_string = ark_string.replace('nan', '"nan"')
        # Safely evaluate the string to a Python list
        ark_list = ast.literal_eval(ark_string)
        # Replace 'nan' strings with np.nan
        return [np.nan if item == 'nan' else item for item in ark_list]
    except (ValueError, SyntaxError) as e:
        print(f"Error: {e}")
        # Return an empty list if there's an issue with conversion
        return ['error_parsing']

In [93]:
# apply the previous function
combined_sessions['Ark_list'] = combined_sessions['Ark'].apply(convert_ark_string)

In [94]:
combined_sessions.head(2)

,session_id,Ark,Date,Referrer,search_terms,first_referrer,length_minutes,visibility,min_visibility,mean_visibility,min_first_3,mean_first_3,min_last_3,mean_last_3,variation_min_vis,variation_mean_vis,Ark_list
0,1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['2016-02-14 21:40:58+01:00', '2016-02-14 21:5...","['-', 'http://gallica.bnf.fr/assets/static/sty...","['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",://,18.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,1640.487421,0,0.0,453,8244.333333,453,8244.333333,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,2,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['2016-02-14 20:24:19+01:00', '2016-02-14 20:2...",['http://bibliotheque.clermont-universite.fr/r...,"['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",http://bibliotheque.clermont-universite.fr,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,3358.471698,0,0.0,222,334.500000,222,334.500000,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


We keep only the sessions where at least one document was consulted.

In [95]:
# remove sessions with no ark
def invalid_arks(ark_list):
    all_nan = all(pd.isna(item) for item in ark_list)
    return all_nan

In [96]:
filtered_sessions = combined_sessions[~combined_sessions['Ark_list'].apply(invalid_arks)]

In [97]:
filtered_sessions.shape

(1181190, 17)

In [98]:
filtered_sessions.head(2)

,session_id,Ark,Date,Referrer,search_terms,first_referrer,length_minutes,visibility,min_visibility,mean_visibility,min_first_3,mean_first_3,min_last_3,mean_last_3,variation_min_vis,variation_mean_vis,Ark_list
0,1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['2016-02-14 21:40:58+01:00', '2016-02-14 21:5...","['-', 'http://gallica.bnf.fr/assets/static/sty...","['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",://,18.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,1640.487421,0,0.0,453,8244.333333,453,8244.333333,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,2,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['2016-02-14 20:24:19+01:00', '2016-02-14 20:2...",['http://bibliotheque.clermont-universite.fr/r...,"['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",http://bibliotheque.clermont-universite.fr,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,3358.471698,0,0.0,222,334.500000,222,334.500000,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [99]:
percentage_full = filtered_sessions.shape[0] / combined_sessions.shape[0] * 100
print(percentage_full, "% of sessions have arks.")

83.35644497136627 % of sessions have arks.


In [100]:
filtered_sessions.to_csv("data_month/sessions_with_arks.csv")

In [2]:
# create dataframe of arks that were computed, concatenate
arks_already_requested = pd.read_csv("arks_final/arks_non_empty.csv", index_col=0))
arks_already_requested2 = pd.read_csv("arks_temp_month/arks_batch_150000.csv", index_col=0))
arks_already_requested3 = pd.read_csv("arks_temp_month/arks_batch2_180000.csv", index_col=0))
arks_computed = pd.concat([arks_already_requested, arks_already_requested2, arks_already_requested3])

In [3]:
arks_computed

,Unnamed: 0,Ark,Theme,Type
0,0.0,btv1b8459670z,NaN,carte
1,1.0,btv1b59624343,NaN,carte
2,2.0,btv1b53036648q,NaN,carte
3,3.0,btv1b530395328,NaN,carte
4,4.0,btv1b5962468w,NaN,carte
...,...,...,...,...
179995,NaN,btv1b6500162b,NaN,NaN
179996,NaN,btv1b6500165k,NaN,NaN
179997,NaN,btv1b6500167d,NaN,NaN
179998,NaN,btv1b6500185b,NaN,NaN


In [4]:
arks_computed.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
# create a dictionnary for quick lookup later
arks_grouped = arks_computed.groupby('Ark').agg({'Theme':'first', 'Type':'first'}).reset_index()
arks_dict = arks_grouped.set_index('Ark').to_dict(orient='index')

In [31]:
arks_dict

{'?': {'Theme': 49.0, 'Type': 'monographie'},
 'bpt6k100001j': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100002x': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100004p': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100005': {'Theme': 90.0, 'Type': 'fascicule'},
 'bpt6k100011w': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000128': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000175': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k10001h': {'Theme': 90.0, 'Type': 'fascicule'},
 'bpt6k100020v': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100025r': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k10002v': {'Theme': 90.0, 'Type': 'fascicule'},
 'bpt6k1000387': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100040j': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100041x': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000429': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000476': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100050w': {'Theme': 33.0, 'Type

In [32]:
# create a df to save it
arks_df = pd.DataFrame.from_dict(arks_dict, orient='index').reset_index()
arks_df.columns = ['Ark', 'Theme', 'Type']

# save arks_df to a CSV file
arks_df.to_csv('arks_final_month/arks_dict.csv', index=False)

In [160]:
# function to create a list of themes and a list of types from a list of arks
def map_themes_and_types(ark_list, arks_dict):
    themes = []
    types = []
    for ark in ark_list:
        if pd.isna(ark):
            themes.append('no_ark')
            types.append('no_ark')
        else:
            theme_type = arks_dict.get(ark, {'Theme': 'no_data', 'Type': 'no_data'})
            theme = theme_type['Theme'] if pd.notna(theme_type['Theme']) else 'no_dewey_class'
            themes.append(theme)
            types.append(theme_type['Type'])
    return themes, types

In [167]:
# apply the function to create a Series of tuples
themes_types_series = filtered_sessions['Ark_list'].progress_apply(
    lambda ark_list: map_themes_and_types(ark_list, arks_dict)
)

# create a DataFrame from the Series
themes_types_df = pd.DataFrame(themes_types_series.tolist(), columns=['themes', 'types'])

# assign the DataFrame columns to the original DataFrame
filtered_sessions.loc[:, ['themes', 'types']] = themes_types_df

100%|███████████████████████████████| 1181190/1181190 [02:29<00:00, 7905.05it/s]


In [168]:
filtered_sessions.head(10)

,session_id,Ark,Date,Referrer,search_terms,first_referrer,length_minutes,visibility,min_visibility,mean_visibility,...,themes,types,very_long,nb_docs,>=10_docs,top_10%_length,top_5%_length,Ark_map,themesL,typesL
0,1,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['2016-02-14 21:40:58+01:00', '2016-02-14 21:5...","['-', 'http://gallica.bnf.fr/assets/static/sty...","['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",://,18.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,1640.487421,...,"[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...",False,5714,True,False,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar..."
1,2,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['2016-02-14 20:24:19+01:00', '2016-02-14 20:2...",['http://bibliotheque.clermont-universite.fr/r...,"['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",http://bibliotheque.clermont-universite.fr,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,3358.471698,...,"[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...",False,1751,True,False,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar..."
2,3,"['btv1b6951272q', 'btv1b6951275z', 'btv1b69512...","['2016-02-15 22:00:43+01:00', '2016-02-15 23:2...",['http://data.bnf.fr/documents-by-rdt/11900585...,"['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",http://data.bnf.fr,83.0,"[54, 71, 84, 138, 59, 137, 54, 24, 54, 24, 11,...",2,159.437621,...,"[no_dewey_class, no_dewey_class, no_dewey_clas...","[image, image, image, image, monographie, imag...",False,46496,True,True,True,"[""btv1b6951272q"", ""btv1b6951275z"", ""btv1b69512...","[no_data, no_data, no_data, no_data, no_data, ...","[no_data, no_data, no_data, no_data, no_data, ..."
3,4,['bpt6k361547'],"['2016-02-15 21:29:00+01:00', '2016-02-15 21:2...",['https://www.google.dz/'],['[]'],https://www.google.dz,0.0,[95],95,95.000000,...,[944.0],[monographie],False,15,True,False,False,"[""bpt6k361547""]",[no_data],[no_data]
4,5,"['btv1b7741316w', 'btv1b7741310d', 'btv1b77413...","['2016-02-14 22:29:54+01:00', '2016-02-14 22:5...","['-', '-', '-', '-', '-', '-', '-', '-', '-', ...","['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",://,23.0,"[27, 53, 27, 53, 27, 53]",27,40.000000,...,"[no_dewey_class, no_dewey_class, no_dewey_clas...","[image, image, image, image, image, image]",False,102,True,False,False,"[""btv1b7741316w"", ""btv1b7741310d"", ""btv1b77413...","[no_data, no_data, no_data, no_data, no_data, ...","[no_data, no_data, no_data, no_data, no_data, ..."
5,6,"['bpt6k3947158', nan, nan, nan, nan, nan, nan,...","['2016-02-14 23:15:45+01:00', '2016-02-14 23:1...",['http://data.bnf.fr/14754920/marcelle_chadal/...,"['[]', '[]', '[]', '[]', '[]', '[]', '[]', '[]...",http://data.bnf.fr,4.0,"[21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",21,21.000000,...,"[no_dewey_class, no_ark, no_ark, no_ark, no_ar...","[partition, no_ark, no_ark, no_ark, no_ark, no...",False,588,True,False,False,"[""bpt6k3947158"", nan, nan, nan, nan, nan, nan,...","[no_data, no_ark, no_ark, no_ark, no_ark, no_a...","[no_data, no_ark, no_ark, no_ark, no_ark, no_a..."
6,7,"[nan, nan, nan, nan, nan, nan, 'bpt6k411269d',...","['2016-02-14 20:57:01+01:00', '2016-02-14 20:5...",['http://gallica.bnf.fr/services/engine/search...,"[""['MMlle. Arriaza']"", ""['M.Mlle. Arriaza']"", ...",http://gallica.bnf.fr,1.0,"[0, 0, 0, 0, 0, 0, 187, 3, 17, 50, 187, 0, 0, ...",1,151.416667,...,"[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...",False,1513,True,False,False,"[nan, nan, nan, nan, nan, nan, ""bpt6k411269d"",...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar...","[no_ark, no_ark, no_ark, no_ark, no_ark, no_ar..."
7,8,['btv1b90248

In [ ]:
# create additional features
filtered_sessions = filtered_sessions.assign(
    very_long=lambda x: x['length_minutes'] >= 120,
    nb_docs = lambda x: x['Ark_list'].apply(lambda y: len(set(i for i in y if i not in (np.nan, 'nan')))),
    '>=10_docs'=lambda x: x['nb_docs'] >= 10
)

In [118]:
top_10_threshold = filtered_sessions['length_minutes'].quantile(0.9)
top_5_threshold = filtered_sessions['length_minutes'].quantile(0.95)
# create a new column indicating whether each session length is in the top 10%
filtered_sessions.loc[:, 'top_10%_length'] = filtered_sessions['length_minutes'] >= top_10_threshold
# same for top 5%
filtered_sessions.loc[:, 'top_5%_length'] = filtered_sessions['length_minutes'] >= top_5_threshold

In [202]:
# function to count the non nan and valid themes and types
def count_unique_valid_entries(lst):
    # Convert the list to a Series, drop NaN values, and then get the unique values
    unique_values = pd.Series(lst).dropna().unique()
    # Exclude 'no_ark' and 'no_data' from the unique values
    unique_values = [value for value in unique_values if value not in ['no_ark', 'no_data']]
    # Return the length of the filtered unique values
    return len(unique_values)


In [204]:
# apply the function to create 2 new features
with tqdm(total=len(filtered_sessions)) as pbar:
    filtered_sessions['nb_types'] = filtered_sessions['types'].progress_apply(lambda x: count_unique_valid_entries(x))
    pbar.update()
    filtered_sessions['nb_themes'] = filtered_sessions['themes'].progress_apply(lambda x: count_unique_valid_entries(x))
    pbar.update()

  0%|                                | 2/1181190 [09:30<93598:14:32, 285.27s/it]


In [205]:
# add diversity features
filtered_sessions = filtered_sessions.assign(
    diversified=((filtered_sessions['nb_themes'] >= 2) | (filtered_sessions['nb_types'] >= 2)),
    diversified_restrictive=((filtered_sessions['nb_themes'] >= 2) & (filtered_sessions['nb_types'] >= 2)),
    diversified_restrictive_5=((filtered_sessions['nb_themes'] >= 5) & (filtered_sessions['nb_types'] >= 5)),
    diversified_5=((filtered_sessions['nb_themes'] >= 5) | (filtered_sessions['nb_types'] >= 5))
)

In [ ]:
# save enriched sessions
filtered_sessions.to_csv("data_temp_month/enriched_sessions/enriched_sessions.csv")

## 4 : Finding missing arks
Some arks may still be missing. We find out which these are and save them to re-request later.

In [7]:
# get enriched sessions
filtered_sessions = pd.read_csv("data_temp_month/enriched_sessions/enriched_sessions.csv")

In [8]:
# use same arks_dict as before
arks_dict

{'?': {'Theme': 49.0, 'Type': 'monographie'},
 'bpt6k100001j': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100002x': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100004p': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100005': {'Theme': 90.0, 'Type': 'fascicule'},
 'bpt6k100011w': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000128': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000175': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k10001h': {'Theme': 90.0, 'Type': 'fascicule'},
 'bpt6k100020v': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100025r': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k10002v': {'Theme': 90.0, 'Type': 'fascicule'},
 'bpt6k1000387': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100040j': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100041x': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000429': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k1000476': {'Theme': 944.0, 'Type': 'monographie'},
 'bpt6k100050w': {'Theme': 33.0, 'Type

In [9]:
# keep only the rows where theme and type are both not nan
arks_dict_cleaned = {ark: info for ark, info in arks_dict.items() if pd.notna(info['Theme']) and pd.notna(info['Type'])}

In [10]:
len(arks_dict_cleaned)

516476

In [14]:
# find the arks not in the dict but in the sessions
filtered_arks = filtered_sessions['Ark_list'].explode().dropna().loc[lambda x: x != 'nan']

# split the ARK lists and flatten them
ark_lists = filtered_arks.str.replace(r"\bnan\b", "", regex=True).str.split(",\s*")
flat_arks = [ark.strip() for sublist in ark_lists for ark in sublist]

# get unique ARKs
unique_arks = set(flat_arks)

# find ARKs not in arks_dict_clean
arks_not_in_dict = unique_arks.difference(arks_dict_cleaned)


In [15]:
arks_not_in_dict

{'',
 "'bpt6k164711m'",
 "'bpt6k5593684c'",
 "'btv1b9034667m']",
 "'bpt6k8815714t']",
 "'bpt6k63826106'",
 "'btv1b8417949v'",
 "'btv1b6916617b'",
 "'btv1b9033384s'",
 "'bpt6k6140441v'",
 "'btv1b10000128f'",
 "'bpt6k447614d'",
 "'btv1b53066316c'",
 "'bpt6k75754g'",
 "'bpt6k5700867m'",
 "'btv1b84512852'",
 "'bpt6k408406h'",
 "'bpt6k522075b'",
 "'bpt6k8197431'",
 "'btv1b105290607']",
 "['bpt6k56274439']",
 "'btv1b84511921'",
 "'bpt6k6569055t'",
 "'btv1b90086936'",
 "'bpt6k75659124'",
 "'bpt6k75597006'",
 "'btv1b8500312j'",
 "'btv1b90071720'",
 "['bpt6k5406613h'",
 "'bpt6k560687q'",
 "'bpt6k310954v'",
 "'bpt6k54487266'",
 "'btv1b7703319n'",
 "'bpt6k9604971p'",
 "'btv1b8553321t'",
 "'bpt6k6519884x'",
 "'btv1b8437713i'",
 "'bpt6k296811d'",
 "'bpt6k9602510'",
 "'btv1b10508392b'",
 "'bpt6k472464v'",
 "'bpt6k5805332g'",
 "'btv1b53037363h'",
 "'bpt6k6254533d'",
 "'bpt6k490642z'",
 "'bpt6k7039773'",
 "'bpt6k16929t']",
 "'bpt6k536507k'",
 "'bpt6k62263449'",
 "'bpt6k545174k'",
 "'bpt6k5545851x'",
 

In [16]:
len(arks_not_in_dict)

2620884

In [17]:
arks_without_brackets = [ark.strip("[]") for ark in arks_not_in_dict]

In [20]:
arks_without_brackets_df = pd.DataFrame(arks_without_brackets, columns=['Ark'])

In [21]:
# save them to re-request them
arks_without_brackets_df.to_csv("data_temp_month/arks_to_request_last.csv")

## 5 : Finding Rabbit Holes

To find sessions that qualify as rabbit holes, we filter the sessions. First we keep only the end of the long tail of session length, so the ones with top 10% time. Then from these the ones where more than 10 documents were visited. And lastly the ones that are diverse, where the documents visited range over 2 different types or 2 different Dewey classes. We also try with different diversity metrics to see how that influences the percentage of rabbit hole sessions.

In [22]:
# function to filter sessions according to different metrics to keep sessions that could qualify as rabbit holes

def filter_sessions(sessions, top_10_length_col, diversified_col, docs_col, diversity_explanation):
    # filter for top 10% length sessions
    sessions_top10_time = sessions[sessions[top_10_length_col] == True].copy()
    print(f"We keep {len(sessions_top10_time) / len(sessions) * 100:.2f}% of the sessions with the highest time")

    # from the top 10% length, keep only the diversified ones
    sessions_top10_time_diversified = sessions_top10_time[sessions_top10_time[diversified_col] == True].copy()
    print(f"We keep {len(sessions_top10_time_diversified) / len(sessions_top10_time) * 100:.2f}% of the long sessions, which are diversified")
    print(diversity_explanation)

    # from the diversified ones, keep only the ones with more than 10 documents
    sessions_rh_final = sessions_top10_time_diversified[sessions_top10_time_diversified[docs_col] == True].copy()
    print(f"We keep {len(sessions_rh_final) / len(sessions_top10_time_diversified) * 100:.2f}% of the diversified sessions, which have more than 10 documents")

    # calculate the percentage of all sessions that qualify as rabbit holes
    print(f"The sessions that could qualify as rabbit holes constitute {len(sessions_rh_final) / len(sessions) * 100:.2f}% of all the sessions")

    return sessions_rh_final

In [25]:
plain_diversity_explanation = "diversfied means 2 types of documents or more or 2 dewey classes or more"
session_rh_final = filter_sessions(filtered_sessions, 'top_10%_length', 'diversified', '>=10_docs', plain_diversity_explanation)             

We keep 10.01% of the sessions with the highest time
We keep 28.99% of the long sessions, which are diversified
diversfied means 2 types of documents or more or 2 dewey classes or more
We keep 99.41% of the diversified sessions, which have more than 10 documents
The sessions that could qualify as rabbit holes constitute 2.89% of all the sessions


#### Tests to see if 2.89% is a reasonable percentage for the rabbit holes sessions

With a more restrictive diversity metric : 2 types AND 2 Dewey classes instead of 'or'

In [26]:
restrictive_diversity_explanation = "diversfied means 2 types of documents or more AND 2 dewey classes or more"
session_rh_test = filter_sessions(filtered_sessions, 'top_10%_length', 'diversified_restrictive', '>=10_docs', restrictive_diversity_explanation)             

We keep 10.01% of the sessions with the highest time
We keep 21.10% of the long sessions, which are diversified
diversfied means 2 types of documents or more AND 2 dewey classes or more
We keep 99.44% of the diversified sessions, which have more than 10 documents
The sessions that could qualify as rabbit holes constitute 2.10% of all the sessions


Then even more restrictive : 5 types and 5 Dewey classes

In [28]:
restrictive5_diversity_explanation = "diversfied means 5 types of documents or more AND 5 dewey classes or more"
session_rh_test = filter_sessions(filtered_sessions, 'top_10%_length', 'diversified_restrictive_5', '>=10_docs', restrictive5_diversity_explanation)             

We keep 10.01% of the sessions with the highest time
We keep 1.93% of the long sessions, which are diversified
diversfied means 5 types of documents or more AND 5 dewey classes or more
We keep 99.17% of the diversified sessions, which have more than 10 documents
The sessions that could qualify as rabbit holes constitute 0.19% of all the sessions


Final test with 5 types or 5 Dewey classes. We see that we get back to about 5 percent of the sessions.

In [29]:
diversity_5_explanation = "diversfied means 5 types of documents or more or 5 dewey classes or more"
session_rh_test = filter_sessions(filtered_sessions, 'top_10%_length', 'diversified_5', '>=10_docs', diversity_5_explanation)             

We keep 10.01% of the sessions with the highest time
We keep 18.51% of the long sessions, which are diversified
diversfied means 5 types of documents or more or 5 dewey classes or more
We keep 99.41% of the diversified sessions, which have more than 10 documents
The sessions that could qualify as rabbit holes constitute 1.84% of all the sessions


We conclude that rabbit holes representing about 2-3% of the sessions is a reasonable number. We save these sessions for later use.

In [30]:
session_rh_final.to_csv("data_temp_month/enriched_sessions/rh_sessions.csv")

The next step will be to compute statistics on both the normal and the rabbit holes sessions.